In [1]:
import xlwings as xw
import time

# 以我們上一堂課的 Excel 檔案爲例，打開你的 stock_portfolio_backtest.xlsx 檔案
wb = xw.Book(r"stock_portfolio_backtest.xlsx")

# 動態截取所有的工作表

wb.sheets 可以搜尋出該工作簿底下所有的工作表，注意它是以 List 的方式回傳：
```python
Sheets([<Sheet [20180915_tw_stock_portfolio.xlsx]TW2882>, <Sheet [20180915_tw_stock_portfolio.xlsx]TW2454>, <Sheet [20180915_tw_stock_portfolio.xlsx]TW2330>])
```

 ```python
# 選擇單一試算表，將該試算表物件存入 sheet 變數
sheet = wb.sheets["TW2330"]

# sheet.activate 代表在 Excel 應用程式上開啓 sheet 函數所代表的試算表物件
sheet.activate()
```

In [4]:
# 透過 wb.sheets 截取到該工作簿/Excel檔案下所有的試算表
sheet = wb.sheets["TW2330"]
sheet.activate()

接下來我們就可以用一個 for 迴圈開啓每一個試算表：

```python
for sheet in wb.sheets:
    # 每迭代到一個試算表，就把它開啓
    sheet.activate()
    # 隔兩秒後，迭代到下一個試算表
    time.sleep(2)
```

In [7]:
for sheet in wb.sheets:
    # 每迭代到一個試算表，就把它開啓
    print(sheet)
    sheet.activate()
    # 隔兩秒後，迭代到下一個試算表
    time.sleep(2)

<Sheet [stock_portfolio_backtest.xlsx]Portfolio>
<Sheet [stock_portfolio_backtest.xlsx]範例>
<Sheet [stock_portfolio_backtest.xlsx]TW2454>
<Sheet [stock_portfolio_backtest.xlsx]TW2330>
<Sheet [stock_portfolio_backtest.xlsx]TW2884>


## 整合一下我們目前學會的...

針對多個試算表進行回測

In [8]:
# 定義一個 run_back_test 函數，處理單一試算表的股價回測
def run_back_test(tsmc_sheet):
    # 從 B1 儲存格開始，往下查找到最後一個有值的儲存格
    last_cell = tsmc_sheet.range('A1').end('down')
    # 截取該儲存格的 row 值
    last_row = last_cell.row

    # 設定我們的範例試算表上的名稱
    tsmc_sheet.range('K2:K11').name = 'weight10d'
    tsmc_sheet.range('K2:K6').name = 'weight5d'

    # 5日加權移動平均計算
    for i in range(6, last_row+1):
        # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
        formula = f"=SUM(B{i-4}:B{i}*weight5d)/SUM(weight5d)"
        # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
        tsmc_sheet.range(f"C{i}").formula_array = formula

    # 10日加權移動平均計算
    for i in range(11, last_row+1):
        # 由於我們需要把兩個陣列相乘，因此這是一個 Excel 的陣列運算
        formula = f"=SUM(B{i-9}:B{i}*weight10d)/SUM(weight10d)"
        # 若一個 Excel 的公式使用到陣列運算，需要用 .formula_array 設定
        tsmc_sheet.range(f"D{i}").formula_array = formula

    # 計算第一天的交易 2017/10/20
    tsmc_sheet.range("E11").value = 1000
    tsmc_sheet.range("F11").value = 0
    tsmc_sheet.range("G11").value = 1000
    tsmc_sheet.range("H11").value = tsmc_sheet.range("L18").value - tsmc_sheet.range("B11").value * 1000
    tsmc_sheet.range("I11").value = tsmc_sheet.range("H11").value +  tsmc_sheet.range("B11").value * tsmc_sheet.range("G11").value


    # 實作交易策略
    for i in range(12, last_row+1):
        # 截取當天的 5日加權移動平均
        short_term_ma = tsmc_sheet.range(f"C{i}").value
        # 截取當天的 10日加權移動平均
        long_term_ma = tsmc_sheet.range(f"D{i}").value
        # 截取當天收盤價
        price_today = tsmc_sheet.range(f"B{i}").value
        # 若 5日 > 10日，而且我有足夠買入以今日收盤價計價的 1000 股的現金，就買入 1000 股（在 E 欄顯示 1000）
        if (short_term_ma > long_term_ma) and (tsmc_sheet.range(f"H{i-1}").value > price_today * 1000):
            tsmc_sheet.range(f"E{i}").value = 1000
        else:
            # 若上述條件不符和，就買入 0 股，（在 E 欄顯示 0）
            tsmc_sheet.range(f"E{i}").value = 0
        # 若 10日 > 5日，而且昨天的持有股數大於 1000 股，就賣出 1000 股
        if (long_term_ma > short_term_ma) and (tsmc_sheet.range(f"G{i-1}").value >= 1000):
            tsmc_sheet.range(f"F{i}").value = 1000
        else:
            tsmc_sheet.range(f"F{i}").value = 0
        # 持有股數，算法是前一天的持有股數 + 今天的買入股數 - 今天的賣出股數
        tsmc_sheet.range(f"G{i}").value = tsmc_sheet.range(f"G{i-1}").value + tsmc_sheet.range(f"E{i}").value - tsmc_sheet.range(f"F{i}").value
        # 持有資金，算法是前一天的持有資金 + 今日收盤價 x (今天的賣出股數 - 今天的買入股數)
        tsmc_sheet.range(f"H{i}").value = tsmc_sheet.range(f"H{i-1}").value + price_today * (tsmc_sheet.range(f"F{i}").value - tsmc_sheet.range(f"E{i}").value)
        # 總資產則是持有股數 x 今日收盤價 + 今日持有資金
        tsmc_sheet.range(f"I{i}").value = tsmc_sheet.range(f"H{i}").value + tsmc_sheet.range(f"G{i}").value * price_today

    # 計算并且將總收益顯示在 L20
    tsmc_sheet.range("L20").value = tsmc_sheet.range(f"I{last_row}").value - tsmc_sheet.range("L18").value

In [9]:
balance = 0

for sheet in wb.sheets:
    # 避免在 Portfolio 試算表上執行回測
    if sheet.name not in ["Portfolio", "範例"]:
        print(sheet.name)
        sheet.activate()
        run_back_test(sheet)
        balance += sheet.range("L20").value
    
print("投資組合收益： ${}".format(balance))

TW2454
TW2330
TW2884
投資組合收益： $126650.0


## 將我們上一課的爬蟲封裝成一個函數



In [10]:
import requests
from bs4 import BeautifulSoup

def yahoo_stock_crawler(stock_id):
    doc = requests.get(f"https://tw.stock.yahoo.com/q/q?s={stock_id}")
    html = BeautifulSoup(doc.text, 'html.parser')
    # 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
    table = html.findAll(text="個股資料")[0].parent.parent.parent
    # 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
    data_row = table.select("tr")[1].select("td")

    # 回傳一個字典
    return {
        "open": data_row[8].text,
        "high": data_row[9].text,
        "low": data_row[10].text,
        "close": data_row[2].text,
        "lastClose": data_row[7].text
    }

In [11]:
# 
import xlwings as xw
from bs4 import BeautifulSoup
import requests
import time

stocks = [2330, 2454, 2884]

wb = xw.Book(r"stock_portfolio_backtest.xlsx")

date = time.strftime("%Y/%m/%d")

for stock in stocks:
    data = yahoo_stock_crawler(stock)
    sheet = wb.sheets["TW{}".format(stock)]
    sheet.activate()
    last_row = sheet.range("B1").end("down").row
    sheet.range(f"B{last_row+1}").value = data["close"]
    sheet.range(f"A{last_row+1}").value = date
    time.sleep(3)

## 但是每一次要手動新增一個試算表很麻煩...

我們希望能夠新增以下功能：

1. 將需要關注的股票記錄在 Portoflio 試算表内
2. 執行時，根據使用者填寫在在 **Portoflio** 試算表内的股票代號到 Yahoo 奇摩上截取該股票的收盤價
3. 若需要多關注一支股票，只需要將股票代號填入 Portoflio 試算表上，讓程式自動建立試算表

## 動態新增試算表

可以在 `wb.sheets` 這個集合下使用 `.add()` 方法：

```python
wb.sheets.add(name="試算表名稱", after=工作表物件, before=工作表物件)
```
**name**: 代表新建立的試算表名稱

**before**/**after**: 代表新建立的試算表的順序是在某個試算表之前或之後

In [19]:
# 新增一個名爲 TW2884 的試算表，放在最右邊
# wb.sheets.add()
sheet = wb.sheets.add(name="TW2882", after=wb.sheets[-1])

In [20]:
# 為新的試算表加上表頭，複製 A1:L20
sheet.range("A1:L20").value = wb.sheets["範例"].range("A1:L20").value

## 問題是要如何判斷何時應該建立新的試算表？

使用 `try...except`：

```python
try:
    來嘗試執行一段程式碼...
except:
    若嘗試執行的程式碼出錯，就執行另一段程式碼
```

我們程式的流程就是：

1. 用爬蟲截取資料
2. 打開相對應的試算表，將寫入資料
3. 若該試算表不存在，python 就會出錯
4. 若出錯，就建立一個新的試算表，再將資料寫入

In [ ]:
# 這段可以幫我們確保stocks[]中的sheet都存在
import xlwings as xw
from bs4 import BeautifulSoup
import requests
import time
import numpy as np

stocks = [2330, 2454, 2882, 2884]

wb = xw.Book(r"stock_portfolio_backtest.xlsx")

date = time.strftime("%Y/%m/%d")

for stock in stocks:
    data = yahoo_stock_crawler(stock)
    
    try: 
        # 若有該試算表，就開啓它，並且寫入資料
        sheet = wb.sheets["TW{}".format(stock)]
        sheet.activate()
        last_row = sheet.range("B1").end("down").row
        sheet.range(f"B{last_row+1}").value = data["close"]
        sheet.range(f"A{last_row+1}").value = date
    except:
        # if not exist,建立一個新的試算表
        sheet = wb.sheets.add(name="TW{}".format(stock), after=wb.sheets[-1])
        # 將範例工作表内的表頭、權重等資料複製到新的工作表
        sheet.range("A1:L20").value = wb.sheets["範例"].range("A1:L20").value
        # 寫入資料
        sheet.range("B2").value = data["close"]
        sheet.range("A2").value = date
    time.sleep(3)

## 但是，我們把需要觀察的股票清單寫死在程式碼不是一件聰明的事

意味著需要經常性的修改程式碼，增加程式出錯的風險

我們乾脆就把要記錄的股票清單記錄在工作表内，讓 Python 每次在執行時會根據記錄在清單内的股票去爬取資料

In [23]:
# 截取 portfolio 工作表下所有的股價代號
port_sheet = wb.sheets["Portfolio"]

last_row = port_sheet.range("A3").end("down").row
# 截取所有的股價代號 A2~Ax
stocks = port_sheet.range(f"A2:A{last_row}").value
# 使用串列表達式將所有的代號從浮點數轉換成整數 list comprehension
stocks = [int(stock) for stock in stocks]
stocks

[2330, 2454, 2884, 2317]

# 大量資料的運算

爬下來的資料，總是需要做分析和運算

意味著我們要學會如何有效的去處理資料

---
# 回顧一下串列

之前我們學會了串列（List）

今天若我需要對串列做 Element-wise（逐元）運算

也就是將串列内的每一個元素都做一樣的運算或操作

必須要用迴圈處理

---
# 範例

將 `km_list` 内的每一個長度從公里換算成英里：

```python
km_list = [3, 5, 10, 21, 42.195]
mile_list = []

km_to_mile = 0.621371192

for km in km_list:
    mile_list.append(km * km_to_mile)

print(mile_list)
# [1.864113576, 3.10685596, 6.21371192, 13.048795032000001, 26.21875744644]
```

---
# for 迴圈雖然可以幫我們做逐元運算

但是程式碼還是過渡冗長...

---
# 串列生成式（List Comprehension）

今天我們知道上述範例最後計算的結果會是一個串列（需要封裝多筆資料）

因此，Python 就提供了串列生成式，允許開發者用簡單的程式碼做出逐元運算

---
# 串列生成式（List Comprehension）

```python
[運算式 for 項目 in 可迭代項目]
```

---
# 串列生成式（List Comprehension）

```python
km_list = [3, 5, 10, 21, 42.195]
km_to_mile = 0.621371192
# 將 km_list 的每一筆資料逐個放入 km，在 for 左邊的運算式算出結果之後放入 mile_list
mile_list = [km * km_to_mile for km in km_list]
print(mile_list)
# [1.864113576, 3.10685596, 6.21371192, 13.048795032000001, 26.21875744644]
```

---

## 接下來...

請在 Portoflio 工作表内新增一筆股票：2882


In [24]:
import xlwings as xw
from bs4 import BeautifulSoup
import requests
import time
import numpy as np

# 開啓 Excel
wb = xw.Book(r"stock_portfolio_backtest.xlsx")

# 截取所有記錄在 Portfolio 試算表内的股票代號
port_sheet = wb.sheets["Portfolio"]
last_row = port_sheet.range("A3").end("down").row
stock_data = port_sheet.range(f"A2:A{last_row}").value
stocks = [int(s) for s in stock_data]

# 產生當日日期
date = time.strftime("%Y/%m/%d")

for stock in stocks:
    data = yahoo_stock_crawler(stock)
    
    try: 
        # 若有該試算表，就開啓它，並且寫入資料
        sheet = wb.sheets["TW{}".format(stock)]
        sheet.activate()
        last_row = sheet.range("B1").end("down").row
        sheet.range(f"B{last_row+1}").value = data["close"]
        sheet.range(f"A{last_row+1}").value = date
    except:
        # 建立一個新的試算表
        sheet = wb.sheets.add(name="TW{}".format(stock), after=wb.sheets[-1])
        # 將範例工作表内的表頭、權重等資料複製到新的工作表
        sheet.range("A1:L20").value = wb.sheets["範例"].range("A1:L20").value
        # 寫入資料
        sheet.range("B2").value = data["close"]
        sheet.range("A2").value = date
    time.sleep(3)

# 記錄總收益    
balance = 0
# 迭代試算表，針對每一個試算表進行回測，將每一個股票的總收益 (L20儲存格) 加總起來
for sheet in wb.sheets:
    sheet.activate()
    last_row = sheet.range("B1").end("down").row
    print(f"{sheet.name} last row: {last_row}")
    run_back_test(sheet)
    balance += sheet.range("L20").value

print("投資組合收益： ${}".format(balance))

ValueError: Sheet named 'TW2882' already present in workbook

## 我們的程式似乎發生了錯誤...

原因是 `xlwings` 的 `.end()` 方法在偵測只有一到兩個 row 的資料工作表會偵測成 `1048576`

![](https://www.dropbox.com/s/mzjuzfypnd7hb8i/end_no_last_row.PNG?dl=1)



# 此時，我們的程式就需要一個錯誤處理的機制

```python
try...except
```


```python
for stock in stocks:
    closing_price = tw_stock_scraper(stock)
    
    try: 
        # 若有該試算表，就開啓它，並且寫入資料
        sheet = wb.sheets["TW{}".format(stock)]
        sheet.activate()
        last_row = sheet.range("B1").end("down").row
        sheet.cells(last_row+1, "B").value = closing_price
        sheet.cells(last_row+1, "A").value = date
    except:
        # 建立一個新的試算表
        sheet = wb.sheets.add(name="TW{}".format(stock), after=wb.sheets[-1])
        # 為新的試算表加上表頭，複製 A1:I1
        sheet.range("A1:I1").value = wb.sheets["TW2330"].range("A1:I1").value
        # 複製回測需要用的權重
        sheet.range("K1:K11").value = np.array(wb.sheets["TW2330"].range("K1:K11").value).reshape(11, 1)
        # 寫入資料
        sheet.cells(2, "B").value = closing_price
        sheet.cells(2, "A").value = date
    time.sleep(3)
```

In [25]:
import xlwings as xw
from bs4 import BeautifulSoup
import requests
import time
import numpy as np

# 開啓 Excel
wb = xw.Book(r"stock_portfolio_backtest.xlsx")

# 截取所有記錄在 Portfolio 試算表内的股票代號
port_sheet = wb.sheets["Portfolio"]
last_row = port_sheet.range("A3").end("down").row
stock_data = port_sheet.range(f"A2:A{last_row}").value
stocks = [int(s) for s in stock_data]

# 產生當日日期
date = time.strftime("%Y/%m/%d")

for stock in stocks:
    data = yahoo_stock_crawler(stock)
    
    try: 
        # 若有該試算表，就開啓它，並且寫入資料
        sheet = wb.sheets["TW{}".format(stock)]
        sheet.activate()
        last_row = sheet.range("B1").end("down").row
        sheet.range(f"B{last_row+1}").value = data["close"]
        sheet.range(f"A{last_row+1}").value = date
    except:
        # 建立一個新的試算表
        sheet = wb.sheets.add(name="TW{}".format(stock), after=wb.sheets[-1])
        # 將範例工作表内的表頭、權重等資料複製到新的工作表
        sheet.range("A1:L20").value = wb.sheets["範例"].range("A1:L20").value
        # 寫入資料
        sheet.range("B2").value = data["close"]
        sheet.range("A2").value = date
    time.sleep(3)

# 記錄總收益    
balance = 0
# 迭代試算表，針對每一個試算表進行回測，將每一個股票的總收益 (L20儲存格) 加總起來
for sheet in wb.sheets:
    sheet.activate()
    last_row = sheet.range("B1").end("down").row
    print(f"{sheet.name} last row: {last_row}")
    # 工作表名稱不是 "Portfolio" 或 "範例"，而且最後一行不是 1048576，也不能小於 11
    if sheet.name not in ["Portfolio", "範例"] and (last_row != 1048576 and last_row > 11):
        print(f"{sheet.name}")
        run_back_test(sheet)
        balance += sheet.range("L20").value

print("投資組合收益： ${}".format(balance))

Portfolio last row: 1048576
範例 last row: 1048576
TW2454 last row: 21
TW2454
TW2330 last row: 39
TW2330
TW2884 last row: 29
TW2884
TW2317 last row: 3
TW2882 last row: 2
投資組合收益： $146350.0


## 小結

1. 跨表格運算十分實用，可以透過工作表名稱去動態選擇一個工作表
2. 爲了避免設定的參數被寫死在程式碼，可將**設定的參數（像是 fugle api key）填入工作表，動態的讓 Python 去截取資料，並且根據截取的參數進行運算**
3. 動態新增試算表時，爲了避免出錯，可以使用 `try...except` 機制處理錯誤

# 預告一下我們的課程内容：打造台股即時看板

接下來我們就發揮我們在過去所學，利用 **Excel、Python**、以及股價 **Web API** 來打造一個及台股即時看板： 

完成品影片：https://youtu.be/Gd_bQTRUrzQ

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/Gd_bQTRUrzQ/0.jpg)](https://www.youtube.com/watch?v=Gd_bQTRUrzQ)